# Campus traffic model
## Version 6-5-2020 20:40

* Insertion of lunch breaks has been moved to develop_detailed_schedule().
* Model parameters are indicated by ##### inside the code.
* Pedestrian and car network data updated, with parking lots, dorms, lunch places, and inside rooms.
* Parking lots are also included in the car network.
* Node mapping rules are imported from note-mapping.txt.
* Agents develop detailed schedule from the studentdata file.
* Constructed networks and agents are pickled and then reloaded at each re-initialization.
* Bus stops were introduced and 50% of car commuters were assumed to "park" at bus stops.
* Visualization was revised to change the node size of parking lots and classrooms based on the number of occupants.
* Parking spaces were assigned according to parking capacities.
* Close contacts are monitored and visualized in a density plot (currently only among moving pedestrians).
* Close contact detection was optimized by using neighbor dictionary.

Revisions after meeting with President Stenger (5/30/2020):
* Road and path networks have been updated to correct errors and to include ITC.
* Roads to ITC and food place/parking lots/bus stop at ITC were added.
* Visualization was revised so that node sizes would represent the actual number more accurately.
* Faculty/staff and grad research assistants were also added, following actual employee data.
    * Now nearly 17k agents! ({'UG': 12752, 'GD': 1162, 'GRA': 375, 'FC': 803, 'SF': 1880})
* Individual speed and time sensitivity variations were increased to make simulation results more natural and smooth.
* Walking speed is gradually decreased to 83/2 = 41.5 m/min as a function of the most recent contact density at each location.
* Agents' preference for Marketplace was implemented.
* Parking usage was monitored and model parameters were calibrated accordingly.
    * Overnight parking numbers were included.
    * Students who can't find a parking spot are automatically converted to bus-commuting.
    * Paid parking places were considered more costly in terms of distance (to discourage their use).
* Inside room capacity information was included and utilized for close contact detection.
* Keeping culumative contacts density map and plotting it at the end.
* Density sweep experiment was conducted.
* Shortest path length calculation was improved by dynamic programming.
* Many minor bugs were fixed.
* Frame-by-frame animation was implemented.

In [1]:
import pycxsimulator
from pylab import *
import pandas as pd
import networkx as nx
import csv
import os.path
import pickle
from matplotlib.colors import ListedColormap

In [2]:
class hour_minute:
    def __init__(self, time_text):
        self.hour = int(time_text[:2])
        self.minute = int(time_text[-2:])
    def str(self):
        return ('0' + str(self.hour))[-2:] + ':' + ('0' + str(self.minute))[-2:]
    def increase(self, Dt):
        self.minute += Dt
        while self.minute >= 60:
            self.hour += 1
            self.minute -= 60
    def decrease(self, Dt):
        self.minute -= Dt
        while self.minute < 0:
            self.hour -= 1
            self.minute += 60
    def min_only(self):
        return self.hour*60 + self.minute
    def difference(self, time_text):
        return int(time_text[:2])*60 + int(time_text[-2:]) - self.min_only()

In [3]:
Dt = 1

R = 6378100.
degree = pi / 180

def distance(pos1, pos2):
    lon1, lat1 = pos1[0]*degree, pos1[1]*degree
    lon2, lat2 = pos2[0]*degree, pos2[1]*degree
    x1, y1, z1 = R*cos(lat1)*cos(lon1), R*cos(lat1)*sin(lon1), R*sin(lat1)
    x2, y2, z2 = R*cos(lat2)*cos(lon2), R*cos(lat2)*sin(lon2), R*sin(lat2)
    return norm(array([x1 - x2, y1 - y2, z1 - z2]))

In [4]:
def create_networks():
    global network_car, network_ped, entrances, entrances_probs, bus_stops, bus_probs
    global possible_parking_places, paid_parking_places, dorms, lunch_places, inside_rooms, room_areas
    global inside_rooms_lunch_places
    global node_mapping, office_mapping

    # car road network

    network_car = nx.DiGraph()
    locations_car = pd.read_csv('car_traffic(0604).csv')
    edges_car = pd.read_excel('car_edge list(0604).xlsx')
    for i in range(len(locations_car)):
        network_car.add_node(str(locations_car.iloc[i].node), 
                             pos = (locations_car.loc[i].longitude, locations_car.loc[i].latitude))
    for i in range(len(edges_car)):
        n1, n2 = str(edges_car.iloc[i]['node 1']), str(edges_car.iloc[i]['node 2'])
        d = distance(network_car.nodes[n1]['pos'], network_car.nodes[n2]['pos'])
        network_car.add_edge(n1, n2, distance = d)
        if edges_car.iloc[i]['one/two-way'] == 'two':
            network_car.add_edge(n2, n1, distance = d)

    ##### entrance nodes for car commuters
    entrances = ['vestal parkway 1', 'vestal parkway 2', 'murray hill road 1', 'murray hill road 68', 'west access road 1']
    entrances_probs = [0.2, 0.3, 0.3, 0.15, 0.05]

    ##### bus stops for bus commuters    
    bus_stops = ['west drive 11', 'east drive 23', 'bunn hill access 12', 'east drive 20', 'east drive 11', 
                 'glenn g.bartle drive 59', 'glenn g.bartle drive 55', 'murray hill road 20']
    bus_probs = [589, 3305, 13, 677, 24, 1, 14, 158]
    bus_probs_sum = sum(bus_probs)
    bus_probs = [x/bus_probs_sum for x in bus_probs]

    # pedestrian road network

    network_ped = nx.Graph()
    locations_ped = pd.read_csv('pedestrian_traffic(0601).csv')
    edges_ped = pd.read_excel('pedestrian_edge list (0601).xlsx')
    for i in range(len(locations_ped)):
        network_ped.add_node(str(locations_ped.iloc[i].node), 
                             pos = (locations_ped.loc[i].longitude, locations_ped.loc[i].latitude))
    for i in range(len(edges_ped)):
        n1, n2 = str(edges_ped.iloc[i]['node1']), str(edges_ped.iloc[i]['node2'])
        d = distance(network_ped.nodes[n1]['pos'], network_ped.nodes[n2]['pos'])
        network_ped.add_edge(n1, n2, distance = d)

    possible_parking_places = [n1 for n1 in network_ped.nodes if n1[:3] == 'lot' and \
                               n1 not in ['lot k', 'lot v', 'lot others1', 'lot others2', 'lot others3', 'lot others4']]
    paid_parking_places = ['lot parking garage', 'lot w', 'lot a1', 'lot b1', 'lot m5', 'lot q2', 
                           'lot visitors parking-1', 'lot visitors parking-2', 'lot visitors parking-3', 
                           'lot visitors parking-4']

    # merging car network into pedestrian network

    edges_to_be_added = []
    for n1 in network_car.nodes:
        n2 = min(network_ped.nodes, 
                 key=lambda x:distance(network_ped.nodes[x]['pos'], network_car.nodes[n1]['pos']))
        d = distance(network_car.nodes[n1]['pos'], network_ped.nodes[n2]['pos'])
        if d < 50: #####
            edges_to_be_added.append([n1, n2, d])
    for n1, n2, d in edges_to_be_added:
        network_ped.add_node(n1, pos = network_car.nodes[n1]['pos'])
        network_ped.add_edge(n1, n2, distance = d)
    for n1, n2 in network_car.edges:
        network_ped.add_node(n1, pos = network_car.nodes[n1]['pos'])
        network_ped.add_node(n2, pos = network_car.nodes[n2]['pos'])
        network_ped.add_edge(n1, n2, distance = network_car.edges[n1, n2]['distance'])

    network_ped = max(nx.connected_component_subgraphs(network_ped), key=len)

    # merging parking lots into car network

    edges_to_be_added = []
    for n1 in possible_parking_places:
        n2 = min(network_car.nodes, 
                 key=lambda x:distance(network_ped.nodes[n1]['pos'], network_car.nodes[x]['pos']))
        d = distance(network_ped.nodes[n1]['pos'], network_car.nodes[n2]['pos'])
        edges_to_be_added.append([n1, n2, d])
    for n1, n2, d in edges_to_be_added:
        network_car.add_node(n1, pos = network_ped.nodes[n1]['pos'])
        network_car.add_edge(n1, n2, distance = d)
        network_car.add_edge(n2, n1, distance = d)

    # identifying dorms and lunch places

    dorms = [n1 for n1 in network_ped.nodes if n1[:4] == 'dorm']
    lunch_places = [n1 for n1 in network_ped.nodes if n1[:4] == 'food']
    inside_rooms = [str(n1) for n1 in pd.read_excel('nodes inside building(0601).xlsx').node]
    inside_rooms_lunch_places = list(set(inside_rooms).union(lunch_places))
    
    # initializing occupancy counters in parking, inside rooms and lunch places
    # (and setting capacity and overnight occupancy for parking nodes)
    
    for n1 in possible_parking_places + inside_rooms_lunch_places:
        network_ped.nodes[n1]['occupancy'] = 0

    f = open('parking-capacities.csv', 'r')
    fcsv = csv.reader(f)
    for row in fcsv:
        while '' in row:
            row.remove('')
        capacity = int(row[0])
        overnight = int(row[1])
        lots = row[2:]
        num_capacity_per_lot = int(capacity / len(lots))
        num_overnight_per_lot = int(overnight / len(lots))
        for lot in lots[:-1]:
            network_ped.nodes[lot]['capacity'] = num_capacity_per_lot
            network_ped.nodes[lot]['occupancy'] = num_overnight_per_lot
            network_ped.nodes[lot]['original-occupancy'] = network_ped.nodes[lot]['occupancy']
        network_ped.nodes[lots[-1]]['capacity'] = capacity - num_capacity_per_lot * len(lots[:-1])
        network_ped.nodes[lots[-1]]['occupancy'] = overnight - num_overnight_per_lot * len(lots[:-1])
        network_ped.nodes[lots[-1]]['original-occupancy'] = network_ped.nodes[lots[-1]]['occupancy']
    f.close()

    # importing node-mapping and office-mapping rules

    def remove_quotes(s):
        return s[1 if s[0] in ["'", '"'] else 0 : -1 if s[-1] in ["'", '"'] else None]

    node_mapping = {}
    with open('node-mapping.txt', 'r') as f:
        fcsv = csv.reader(f)
        for row in fcsv:
            room_node = remove_quotes(row[0])
            for room_label in row[1:]:
                if room_label.strip() != '':
                    node_mapping[remove_quotes(room_label.strip())] = room_node

    office_mapping = {}
    with open('office-mapping.txt', 'r') as f:
        fcsv = csv.reader(f)
        for row in fcsv:
            office_mapping[remove_quotes(row[0].strip())] = remove_quotes(row[1].strip())
            
    # importing room areas
    
    room_areas_data = pd.read_excel('inside-room-capacities.xls')
    room_areas_unaggregated = {str(room_areas_data.iloc[i].BUILDING) + str(room_areas_data.iloc[i].ROOM):
                                    float(room_areas_data.iloc[i].area) for i in range(len(room_areas_data))}
    room_areas = {}
    for x in node_mapping:
        if x in room_areas_unaggregated:
            node_name = node_mapping[x]
            if node_name in room_areas:
                room_areas[node_name] += room_areas_unaggregated[x] * 0.092903 # converting sqft to m^2
            else:
                room_areas[node_name] = room_areas_unaggregated[x] * 0.092903 # converting sqft to m^2

    # following are just estimates
    room_areas['food-glenn'] += 1200 * 0.092903 # converting sqft to m^2 
    room_areas['food-biotechnology building-3'] = 600 * 0.092903 # converting sqft to m^2 
    room_areas['food-science1'] = 1600 * 0.092903 # converting sqft to m^2 
    room_areas['food1-market place'] = 6000 * 0.092903 # converting sqft to m^2 
    room_areas['food2-market place'] = 6000 * 0.092903 # converting sqft to m^2 
    room_areas['food-uu'] = 1200 * 0.092903 # converting sqft to m^2

In [5]:
path_length_dict = {}

def path_length(n1, n2):
    global network_ped, path_length_dict
    node_pair = tuple(sorted([n1, n2]))
    if node_pair in path_length_dict:
        return path_length_dict[node_pair]
    else:
        d = nx.shortest_path_length(network_ped, n1, n2, weight = 'distance')
        path_length_dict[node_pair] = d
        return d

In [6]:
def lunch_break_time(sch):
    long_breaks = [(sch[i][1], sch[i+1][0]) for i in range(len(sch)-1) 
                    if hour_minute(sch[i][1]).difference(sch[i+1][0]) >= 30] #####
    if len(long_breaks) == 0:
        return None
    best_break = min(long_breaks, key=lambda x:abs(hour_minute(x[0]).difference('12:00'))) #####
    best_break_time = hour_minute(best_break[0])
    best_break_time.increase(best_break_time.difference(best_break[1])//2)
    if best_break_time.difference('10:00') > 0 or best_break_time.difference('19:00') < 0: #####
        return None
    return best_break_time.str()

def reduced_speed_by_congestion(sp, pos):
    global most_recent_contacts_density_map
    x, y = discrete_location(pos)
    d = most_recent_contacts_density_map[y, x]
    return sp * exp(-d/2.0) + 41.5 * (1 - exp(-d/2.0)) ##### exponentially decreasing speed along contacts density d

class human:
    def __init__(self, anid, category, home, office = None, schedule = None, edge = None, edge_frac = 0., 
                 moving = False, driving = False, car = None, active = False):
        self.anid = anid
        self.speed = 83. + normal(0, 4) ##### m/min.
        self.category = category
        self.home = home
        self.office = office
        self.node = home
        self.schedule = schedule if schedule != None else []
        self.detailed_schedule = []
        self.edge = edge
        self.edge_frac = edge_frac
        self.moving = moving
        self.driving = driving
        self.car = car
        self.active = active
        self.left = False
        
    def location(self):
        net = network_car if self.driving else network_ped
        if self.node != None:
            return net.nodes[self.node]['pos']
        else:
            n1, n2 = self.edge
            return array(net.nodes[n1]['pos'])*(1-self.edge_frac) + \
                   array(net.nodes[n2]['pos'])*self.edge_frac

    # the following is no longer used; only for initial testing and demos
    '''
    def generate_random_schedule(self):
        if self.car != None:
            self.home = choice(entrances, p = entrances_probs)
            self.node = self.home
            self.car.node = self.home
        else:
            self.home = choice(dorms)
            self.node = self.home
        t = hour_minute('08:00') #####
        for i in range(randint(2, 5)): #####
            t.increase(randint(30, 180)) #####
            t1 = t.str()
            t.increase(randint(30, 180)) #####
            t2 = t.str()
            while True:
                destination = choice(network_ped.nodes)
                if destination not in dorms and destination not in lunch_places:
                    break
            self.schedule.append([t1, t2, destination])
            if t.difference('19:00') <= 0: #####
                break
    '''
    
    def develop_detailed_schedule(self):
        
        self.schedule.sort(key=lambda x:x[0])

        # inserting lunch break
        if self.category in ['UG', 'GD', 'GRA'] or (self.category in ['FC', 'SF'] and random() < 0.2): #####
            lunch_time = lunch_break_time(self.schedule)
            if lunch_time != None:
                lunch_time = hour_minute(lunch_time)
                t1 = lunch_time.str()
                lunch_time.increase(5) #####
                t2 = lunch_time.str()
                self.schedule.append([t1, t2, '***lunch***'])
                self.schedule.sort(key=lambda x:x[0])
                i = [x[-1] for x in self.schedule].index('***lunch***')
                lp1 = None
                if i > 0:
                    lp1 = min(lunch_places, 
                              key=lambda x:path_length(x, self.schedule[i-1][-1]) * \
                                           (0.3 if x[-12:] == 'market place' else 1)) #####
                lp2 = None
                if i < len(self.schedule) - 1:
                    lp2 = min(lunch_places, 
                              key=lambda x:path_length(x, self.schedule[i+1][-1]) * \
                                           (0.3 if x[-12:] == 'market place' else 1)) #####
                if lp1 == None:
                    self.schedule[i][-1] = lp2
                elif lp2 == None:
                    self.schedule[i][-1] = lp1
                else:
                    if path_length(lp1, self.schedule[i-1][-1]) < path_length(lp2, self.schedule[i+1][-1]):
                        self.schedule[i][-1] = lp1
                    else:
                        self.schedule[i][-1] = lp2
                    
        current_node = self.home
        next_node = self.schedule[0][-1]
        
        if self.car != None:
            need_bus = False
            if (self.category in ['FC', 'SF'] and random() < 1.0) or \
               (self.category in ['GD', 'GRA'] and random() < 0.1) or \
               (self.category in ['UG'] and random() < 0.3): ##### probabilities of own-car commuting
                still_available_parking_places = [x for x in possible_parking_places 
                                                  if network_ped.nodes[x]['occupancy'] < network_ped.nodes[x]['capacity']]
                if len(still_available_parking_places) > 0:
                    parking_node = min([x for x in still_available_parking_places],
                                       key=lambda x:path_length(x, next_node) * \
                                                    ((10 if self.category in ['UG', 'GD', 'GRA'] else 2)
                                                     if x in paid_parking_places else 1)) ##### paid lot distance costs
                    network_ped.nodes[parking_node]['occupancy'] += 1
                else:
                    need_bus = True                    
            else:
                need_bus = True
            if need_bus:
                bus_probs_adjusted = array([bus_probs[i] / path_length(bus_stops[i], next_node)
                                            for i in range(len(bus_stops))])
                bus_probs_adjusted = bus_probs_adjusted / sum(bus_probs_adjusted)
                parking_node = choice(bus_stops, p = bus_probs_adjusted)
            time_needed = int(nx.shortest_path_length(network_car, current_node, parking_node, 
                                                      weight = 'distance') / self.car.speed + \
                              path_length(parking_node, next_node) / self.speed)
            time_needed += randint(10, 40) if self.category in ['UG', 'GD'] and parking_node not in bus_stops \
                                           else randint(3, 8) #####
            time_departure = hour_minute(self.schedule[0][0])
            time_departure.decrease(time_needed)
            route = nx.shortest_path(network_car, current_node, parking_node, weight = 'distance')
            for i in range(len(route) - 1):
                entry = []
                entry.append(time_departure.str() if i == 0 else '00:00')
                entry.append((route[i], route[i+1]))
                if i == len(route) - 2:
                    entry.append('park')
                self.detailed_schedule.append(entry)
            current_node = parking_node
            time_departure = hour_minute('00:00')
        else:
            time_needed = int(path_length(current_node, next_node) / self.speed) + randint(3, 8) #####
            time_departure = hour_minute(self.schedule[0][0])
            time_departure.decrease(time_needed)
        route = nx.shortest_path(network_ped, current_node, next_node, weight = 'distance')            
        for i in range(len(route) - 1):
            self.detailed_schedule.append([time_departure.str() if i == 0 else '00:00', (route[i], route[i+1])])
        
        for j in range(len(self.schedule) - 1):
            current_node = self.schedule[j][-1]
            next_node = self.schedule[j + 1][-1]
            time_departure = hour_minute(self.schedule[j][1])
            time_gap = time_departure.difference(self.schedule[j + 1][0])
            detoured = False
            if self.home in dorms: # if this is an on-campus resident
                if int(path_length(current_node, self.home) / self.speed) + \
                   int(path_length(self.home, next_node) / self.speed) < \
                   time_gap - 60: ##### if there is still one hour left even if temporarily going back home, then deo
                    route = nx.shortest_path(network_ped, current_node, self.home, weight = 'distance')            
                    for i in range(len(route) - 1):
                        self.detailed_schedule.append([time_departure.str() if i == 0 else '00:00', (route[i], route[i+1])])
                    time_departure = hour_minute(self.schedule[j + 1][0])
                    time_needed = int(path_length(self.home, next_node) / self.speed) + randint(3, 8) #####
                    time_departure.decrease(time_needed)
                    route = nx.shortest_path(network_ped, self.home, next_node, weight = 'distance')            
                    for i in range(len(route) - 1):
                        self.detailed_schedule.append([time_departure.str() if i == 0 else '00:00', (route[i], route[i+1])])
                    detoured = True
            if not detoured:
                time_needed = int(path_length(current_node, next_node) / self.speed)
                time_gap = int((time_gap - time_needed) / 2)
                time_departure.increase(time_gap)
                route = nx.shortest_path(network_ped, current_node, next_node, weight = 'distance')            
                for i in range(len(route) - 1):
                    self.detailed_schedule.append([time_departure.str() if i == 0 else '00:00', (route[i], route[i+1])])

        current_node = self.schedule[-1][-1]
        time_departure = hour_minute(self.schedule[-1][1])
        
        if self.car != None:            
            if parking_node in bus_stops:
                bus_probs_adjusted = array([bus_probs[i] / path_length(bus_stops[i], current_node)
                                            for i in range(len(bus_stops))])
                bus_probs_adjusted = bus_probs_adjusted / sum(bus_probs_adjusted)
                parking_node = choice(bus_stops, p = bus_probs_adjusted)
            route = nx.shortest_path(network_ped, current_node, parking_node, weight = 'distance')
            for i in range(len(route) - 1):
                entry = []
                entry.append(time_departure.str() if i == 0 else '00:00')
                entry.append((route[i], route[i+1]))
                if i == len(route) - 2:
                    entry.append('ride')
                self.detailed_schedule.append(entry)
            current_node = parking_node
            time_departure = hour_minute('00:00')

        route = nx.shortest_path(network_car if self.car != None else network_ped,
                                 current_node, self.home, weight = 'distance')
        for i in range(len(route) - 1):
            entry = []
            entry.append(time_departure.str() if i == 0 else '00:00')
            entry.append((route[i], route[i+1]))
            if i == len(route) - 2:
                entry.append('leave')
            self.detailed_schedule.append(entry)
            
    def print_detailed_schedule(self):
        current_node = None
        for move in self.detailed_schedule:
            if move[0] != '00:00':
                if current_node != None:
                    print('')
                print('[', move[0], ']', ':', move[1][0], '->', move[1][1], end='')
            else:
                if move[1][0] != current_node:
                    print('***** ERROR!! Edges are not connected *****')
                else:
                    print(' ->', move[1][1], end='')
            current_node = move[1][1]
            if len(move) == 3:
                print(' **', move[2])
        
    def act(self):
        if self.left:
            return
        if (not self.moving) and time_of_day.difference(self.detailed_schedule[0][0]) <= 0: 
            # time to move?
            self.active = True
            if self.driving:
                self.car.active = True
            self.moving = True
            if self.node in inside_rooms_lunch_places:
                if self.category in ['UG', 'GD'] or (self.node in lunch_places and self.node != self.office):
                    network_ped.nodes[self.node]['occupancy'] -= 1
            self.node = None
            self.edge = self.detailed_schedule[0][1]
            self.edge_frac = 0.

        if self.moving:
            speed = self.car.speed if self.driving else self.speed
            speed = reduced_speed_by_congestion(speed, self.location()) # due to congestion
            net = network_car if self.driving else network_ped
            delta_t = Dt
            while delta_t > 0 and self.moving:
                edge_length = net.edges[self.edge]['distance']
                remaining_distance = edge_length * (1 - self.edge_frac)
                if remaining_distance > speed * delta_t:
                    remaining_distance -= speed * delta_t
                    self.edge_frac = 1 - remaining_distance / edge_length
                    if self.driving:
                        self.car.edge_frac = self.edge_frac
                    delta_t = 0
                else:
                    delta_t -= remaining_distance / speed
                    self.moving = False
                    self.node = self.detailed_schedule[0][1][1]
                    if self.node in inside_rooms_lunch_places:
                        if self.category in ['UG', 'GD'] or (self.node in lunch_places and self.node != self.office):
                            network_ped.nodes[self.node]['occupancy'] += 1
                    self.edge = None
                    self.edge_frac = 0.
                    if self.driving:
                        self.car.node = self.node
                        self.car.edge = None
                        self.car.edge_frac = 0.                    
                    if len(self.detailed_schedule[0]) == 3:
                        command = self.detailed_schedule[0][2]
                        if self.driving and command == 'park':
                            if self.node not in bus_stops:
                                network_ped.nodes[self.node]['occupancy'] += 1
                            self.driving = False
                            net = network_ped
                        elif (not self.driving) and command == 'ride':
                            if self.node not in bus_stops:
                                network_ped.nodes[self.node]['occupancy'] -= 1
                            self.car.node = self.node
                            self.driving = True
                            net = network_car
                        elif command == 'leave':
                            self.active = False
                            self.left = True
                            if self.car != None:
                                self.car.active = False
                            return
                    del self.detailed_schedule[0] # finished traversing one edge
                    if self.detailed_schedule[0][0] == '00:00': # continue moving
                        self.moving = True
                        if self.node in inside_rooms_lunch_places:
                            if self.category in ['UG', 'GD'] or (self.node in lunch_places and self.node != self.office):
                                network_ped.nodes[self.node]['occupancy'] -= 1
                        self.node = None
                        self.edge = self.detailed_schedule[0][1]
                        if self.driving:
                            self.car.node = None
                            self.car.edge = self.edge
    
class car:
    def __init__(self, node, edge = None, edge_frac = 0., active = False):
        self.speed = 536.5 + normal(0, 26) ##### m/min. = 20 mph
        self.node = node
        self.edge = edge
        self.edge_frac = edge_frac
        self.active = False
        
    def location(self):
        net = network_car
        if self.node != None:
            return net.nodes[self.node]['pos']
        else:
            n1, n2 = self.edge
            return array(net.nodes[n1]['pos'])*(1-self.edge_frac) + \
                   array(net.nodes[n2]['pos'])*self.edge_frac

In [7]:
def initialize(monitor = 500, density = 1.0):
    global time_of_day, network_car, network_ped, entrances, entrances_probs, bus_stops, bus_probs
    global possible_parking_places, dorms, lunch_places, inside_rooms, room_areas, node_mapping, office_mapping
    global inside_rooms_lunch_places
    global pedestrians, cars
    global contacts_cumulative, contacts_density_map, most_recent_contacts_density_map
    global contacts_density_resolution, contacts_density_map_cumulative
    global frame_count

    time_of_day = hour_minute('06:30') #####
    frame_count = 0
    
    contacts_cumulative = 0.
    contacts_density_resolution = 200
    contacts_density_map = zeros([contacts_density_resolution, contacts_density_resolution])
    most_recent_contacts_density_map = zeros([contacts_density_resolution, contacts_density_resolution])
    contacts_density_map_cumulative = zeros([contacts_density_resolution, contacts_density_resolution])

    if os.path.isfile('constructed-network-data.pickle'):
        f = open('constructed-network-data.pickle', 'rb')
        network_car, network_ped, entrances, entrances_probs, bus_stops, bus_probs, \
        possible_parking_places, dorms, lunch_places, inside_rooms, room_areas, \
        node_mapping, pedestrians, cars = pickle.load(f)
        f.close()
        inside_rooms_lunch_places = list(set(inside_rooms).union(lunch_places))
        if density < 1.0:
            reduce_pedestrians_density(density)
        return

    pedestrians = []
    cars = []

    ##### the following was a legacy code that was used only for initial testing and demos
    '''
    for i in range(300): #####
        if i % monitor == 0:
            print(i, end=' ')
        h = human(undergrad, 'aaa', home = None)
        pedestrians.append(h)
        if random() < 0.3: #####
            c = car(node = None, edge = h.edge, edge_frac = h.edge_frac)
            cars.append(c)
            h.car = c
            h.driving = True
        h.generate_random_schedule()
        h.develop_detailed_schedule()
    '''

    print('Creating networks...')
    create_networks()

    anid = None
    
    # faculty/staff/research grad students
    print('Reading employee data...')
    
    employeedata = pd.read_excel('employees.xlsx')
    for i in range(len(employeedata)):
        if i % monitor == 0:
            print(i, end=' ')
        
        row = employeedata.iloc[i]
        anid = row.AnID
        if random() < row.fte and row.bldg in office_mapping: # using FTE as the likelihood to be on campus
            h = human(anid = anid, category = row.category, home = choice(entrances, p = entrances_probs))
            h.node = h.home
            pedestrians.append(h)
            c = car(node = h.node, edge = h.edge, edge_frac = h.edge_frac)
            cars.append(c)
            h.car = c
            h.driving = True
            room_node = choice([x for x in set(inside_rooms_lunch_places + dorms) if x.startswith(office_mapping[row.bldg])])
            h.office = room_node
            t1 = '0' + str(randint(7, {'F':10, 'S':8, 'G':10}[row.category[0]])) + ':'+ ('0' + str(randint(60)))[-2:] #####
            t2 = '11:' + ('0' + str(randint(30, 60)))[-2:] #####
            pedestrians[-1].schedule.append([t1, t2, room_node])
            t1 = '12:' + ('0' + str(randint(0, 30)))[-2:] #####
            t2 = str(randint(16, {'F':19, 'S':18, 'G':21}[row.category[0]])) + ':' + ('0' + str(randint(60)))[-2:] #####
            pedestrians[-1].schedule.append([t1, t2, room_node])
    
    # the following was an initial attempt to generate them randomly -- no longer used
    '''
    for i in range(1900): #####
        anid = choice(['Faculty', 'Staff', 'GRA'], p=[750/1900, 500/1900, 650/1900]) + str(i) #####
        h = human(anid = anid, category = {'F':'FC', 'S':'SF', 'G':'GRA'}[anid[0]], 
                  home = choice(entrances, p = entrances_probs))
        h.node = h.home
        pedestrians.append(h)
        c = car(node = h.node, edge = h.edge, edge_frac = h.edge_frac)
        cars.append(c)
        h.car = c
        h.driving = True
        room_node = choice(inside_rooms)
        t1 = '0' + str(randint(7, {'F':9, 'S':8, 'G':8}[anid[0]])) + ':'+ ('0' + str(randint(60)))[-2:] #####
        t2 = '11:' + ('0' + str(randint(30, 60)))[-2:] #####
        pedestrians[-1].schedule.append([t1, t2, room_node])
        t1 = '12:' + ('0' + str(randint(0, 30)))[-2:] #####
        t2 = str(randint(15, {'F':19, 'S':17, 'G':20}[anid[0]])) + ':' + ('0' + str(randint(60)))[-2:] #####
        pedestrians[-1].schedule.append([t1, t2, room_node])
    '''

    # course-taking students
    classdata = pd.read_excel('classes.xls')
    studentdata = pd.read_csv('studentdata-2019only.csv')
    print('\nReading course registration data...')

    for i in range(len(studentdata)):
        if i % monitor == 0:
            print(i, end=' ')
        
        row = studentdata.iloc[i]
        if row.anid != anid:
            if anid != None:
                if len(pedestrians[-1].schedule) == 0:
                    if pedestrians[-1].car != None:
                        del cars[cars.index(pedestrians[-1].car)]
                    del pedestrians[-1]
            
            anid = row.anid
            h = human(anid = anid, category = row.PROGRAM_LEVEL, 
                      home = choice(entrances, p = entrances_probs) if row.ResHall is nan else node_mapping[row.ResHall])
            h.node = h.home
            pedestrians.append(h)
            if row.ResHall is nan:
                c = car(node = h.node, edge = h.edge, edge_frac = h.edge_frac)
                cars.append(c)
                h.car = c
                h.driving = True

        class_hit = classdata[classdata.COURSE_REFERENCE_NUMBER == row.COURSE_REFERENCE_NUMBER]
        if len(class_hit) == 1:
            course = class_hit.iloc[0]
            t1 = str(course.BEGIN_TIME)
            t1 = ('00' + t1)[-4:]
            t1 = t1[:2] + ':' + t1[-2:]
            t2 = str(course.END_TIME)
            t2 = ('00' + t2)[-4:]
            t2 = t2[:2] + ':' + t2[-2:]
            room_node = node_mapping[course.BUILDING + course.ROOM]
            pedestrians[-1].schedule.append([t1, t2, room_node])
            
    if len(pedestrians[-1].schedule) == 0:
        del pedestrians[-1]

    pedestrians.sort(key = lambda x:x.schedule[0][0])

    print('\nBuilding detailed schedules of pedestrians...')
    for i in range(len(pedestrians)):
        if i % int(monitor / 10) == 0:
            print(i, end=' ')
        pedestrians[i].develop_detailed_schedule()

    for n1 in possible_parking_places:
        network_ped.nodes[n1]['occupancy'] = network_ped.nodes[n1]['original-occupancy']
    
    f = open('constructed-network-data.pickle', 'wb')
    pickle.dump([network_car, network_ped, entrances, entrances_probs, bus_stops, bus_probs, \
                 possible_parking_places, dorms, lunch_places, inside_rooms, room_areas, \
                 node_mapping, pedestrians, cars], f)
    f.close()
    
    if density < 1.0:
        reduce_pedestrians_density(density)
        
def reduce_pedestrians_density(density):
    global pedestrians
    i = 0
    while i < len(pedestrians):
        if random() < 1 - density:
            del pedestrians[i]
        else:
            i += 1

In [8]:
plot_range = [-75.983, -75.957, 42.081, 42.0965]

newcolors = cm.get_cmap('Oranges', 256)(linspace(0, 1, 256))
gmax, bmax = newcolors[0][1:3]
for i in range(len(newcolors)):
    newcolors[i, 1] /= gmax
    newcolors[i, 2] /= bmax
newOranges = ListedColormap(newcolors)

def observe():
    global time_of_day, network_car, network_ped, pedestrians, cars, contacts_density_map, contacts_density_resolution
    global plot_range, newOranges
    global animating, animation_folder, frame_count

    cla()
    
    # contacts density map
    
    imshow(contacts_density_map, vmin = 0, vmax = 30, cmap = newOranges, extent = plot_range, origin = 'lower')
    
    # car road network
    nx.draw_networkx_edges(network_car, alpha = 0.2, edge_color = 'c',
                           pos = {i:network_car.nodes[i]['pos'] for i in network_car.nodes},
                           width = 1.5, arrows = False)
    nx.draw_networkx_nodes(network_car, node_size = [6*network_ped.nodes[i]['occupancy']
                                                     if i in possible_parking_places else 0 
                                                     for i in network_car.nodes], node_color = 'b',
                           pos = {i:network_car.nodes[i]['pos'] for i in network_car.nodes},
                           with_labels = False, alpha = 0.5)

    # pedestrian road network
    nx.draw_networkx_edges(network_ped, alpha = 0.2, edge_color = 'g',
                           pos = {i:network_ped.nodes[i]['pos'] for i in network_ped.nodes})
    nx.draw_networkx_nodes(network_ped, node_size = [1.5*network_ped.nodes[i]['occupancy']
                                                     if i in inside_rooms_lunch_places else 0 
                                                     for i in network_ped.nodes], node_color = 'g', 
                           pos = {i:network_ped.nodes[i]['pos'] for i in network_ped.nodes},
                           with_labels = False, alpha = 0.5)
    
    # pedestrians
    # faculty/staff
    xs, ys = [], []
    for p in pedestrians:
        if p.category in ['FC', 'SF'] and p.active and not p.driving and p.node == None:
            loc = p.location()
            xs.append(loc[0])
            ys.append(loc[1])
    if len(xs) > 0:
        plot(xs, ys, 'k.', alpha = 0.5)
    # students
    xs, ys = [], []
    for p in pedestrians:
        if p.category in ['UG', 'GD', 'GRA'] and p.active and not p.driving and p.node == None:
            loc = p.location()
            xs.append(loc[0])
            ys.append(loc[1])
    if len(xs) > 0:
        plot(xs, ys, 'g.', alpha = 0.5)

    # cars
    xs, ys = [], []
    for p in cars:
        if p.active and p.node == None:
            loc = p.location()
            xs.append(loc[0])
            ys.append(loc[1])
    if len(xs) > 0:
        plot(xs, ys, 'bo', alpha = 0.5)

    text(-75.983, 42.095, 'Tuesday ' + time_of_day.str(), fontsize = 16)
    
    axes().set_aspect(1/cos(42*degree))
    axis('off')
    axis(plot_range)
    
    # frame-by-frame animation
    if animating:
        savefig(animation_folder + ('000' + str(frame_count))[-4:] + '.png')
    
def draw_networks_only():
    global network_car, network_ped
    
    axes().set_aspect(1/cos(42*degree))

    # car road network
    nx.draw(network_car, node_size = [100 if i[:3] in ['lot'] else 0 for i in network_car.nodes], 
            pos = {i:network_car.nodes[i]['pos'] for i in network_car.nodes}, node_color = 'b',
            with_labels = False, width = 1.5, arrows = False, alpha = 0.2, edge_color = 'c')

    # pedestrian road network
    nx.draw(network_ped, node_size = [100 if i[:3] in ['foo', 'dor'] or i in inside_rooms else 0 for i in network_ped.nodes], 
            pos = {i:network_ped.nodes[i]['pos'] for i in network_ped.nodes},
            with_labels = False, alpha = 0.2, 
            node_color = [{'dor':'g', 'foo':'r'}[i[:3]] if i[:3] in ['foo', 'dor'] else 'm' if i in inside_rooms else 'k' 
                          for i in network_ped.nodes], edge_color = 'g')
    
def draw_contacts_density_map_cumulative():
    global network_car, network_ped, contacts_density_map_cumulative

    cla()
    
    imshow(contacts_density_map_cumulative, vmin = 0, vmax = 3000, cmap = newOranges, extent = plot_range, origin = 'lower')
    
    nx.draw_networkx_edges(network_car, alpha = 0.2, edge_color = 'c',
                           pos = {i:network_car.nodes[i]['pos'] for i in network_car.nodes},
                           width = 1.5, arrows = False)
    nx.draw_networkx_edges(network_ped, alpha = 0.2, edge_color = 'g',
                           pos = {i:network_ped.nodes[i]['pos'] for i in network_ped.nodes})

    axes().set_aspect(1/cos(42*degree))
    axis('off')
    axis(plot_range)    

In [9]:
#plot_range = [-75.983, -75.957, 42.081, 42.0965]

plot_range_width = plot_range[1] - plot_range[0]
plot_range_height = plot_range[3] - plot_range[2]

def discrete_location(pos):
    x = int((pos[0] - plot_range[0]) / plot_range_width * contacts_density_resolution)
    y = int((pos[1] - plot_range[2]) / plot_range_height * contacts_density_resolution)
    return array([x, y])

def discrete_location_for_neighbor_detection(pos):
    x = int((pos[0] - plot_range[0]) / plot_range_width * 600)
    y = int((pos[1] - plot_range[2]) / plot_range_height * 600)
    return (x, y)

def monitor_contacts():
    global network_car, network_ped, pedestrians, inside_rooms, room_areas
    global contacts_cumulative, contacts_density_map, most_recent_contacts_density_map
    global contacts_density_resolution, contacts_density_map_cumulative

    for x in range(contacts_density_resolution):
        for y in range(contacts_density_resolution):
            most_recent_contacts_density_map[y, x] = 0.

    # contact detection inside rooms
    
    for x in inside_rooms_lunch_places:
        if x in room_areas:
            num_people = network_ped.nodes[x]['occupancy']
            num_within_circle = num_people / room_areas[x] * 2.6268 # (3 ft)^2 x pi in m^2
            if num_within_circle > 1:
                detected_contacts = (num_within_circle - 1) * num_people * 0.05 
                # 10 mins = one case, and each contact is counted twice, so 1/10/2 = 0.05
                contacts_cumulative += detected_contacts
                x, y = discrete_location(network_ped.nodes[x]['pos'])
                contacts_density_map[y, x] += detected_contacts
                most_recent_contacts_density_map[y, x] += detected_contacts
        #else:
        #    if network_ped.nodes[x]['occupancy'] > 0:
        #        print(x, 'needs room area!')

    # contact detection in outside space
    walking_ppl_pos = [x.location() for x in pedestrians if x.active and not x.driving and x.moving]
    neighbor_dict = {}
    for pos in walking_ppl_pos:
        disc_pos = discrete_location_for_neighbor_detection(pos)
        if disc_pos in neighbor_dict:
            neighbor_dict[disc_pos].append(pos)
        else:
            neighbor_dict[disc_pos] = [pos]
    for (x, y) in neighbor_dict:
        neighbors = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if (x+dx, y+dy) in neighbor_dict:
                    neighbors.extend(neighbor_dict[(x+dx, y+dy)])
        for pos in neighbor_dict[(x, y)]:
            already_skipped_self = False
            for nb_pos in neighbors:
                if not (pos == nb_pos).all() or ((pos == nb_pos).all() and already_skipped_self):
                    if distance(pos, nb_pos) < 1.8288: ##### 6 feet distance
                        contacts_cumulative += 0.05 # 10 mins = one case, and each contact is counted twice, so 1/10/2 = 0.05
                        x, y = discrete_location((pos + nb_pos) / 2)
                        contacts_density_map[y, x] += 0.05
                        most_recent_contacts_density_map[y, x] += 0.05
                if (pos == nb_pos).all():
                    already_skipped_self = True
                
    # diffusion
    next_contacts_density_map = copy(contacts_density_map)
    for x in range(contacts_density_resolution):
        for y in range(contacts_density_resolution):
            for dx in range(-1 if x > 0 else 0, 2 if x < contacts_density_resolution - 1 else 1):
                for dy in range(-1 if y > 0 else 0, 2 if y < contacts_density_resolution - 1 else 1):
                    next_contacts_density_map[y, x] += (contacts_density_map[y+dy, x+dx]-contacts_density_map[y, x])*0.1 #####
    contacts_density_map = next_contacts_density_map

    # exponential decay
    contacts_density_map *= 0.9 #####

    # adding to cumulative map
    contacts_density_map_cumulative += contacts_density_map

In [10]:
parking_usage_records = {}
monitoring_parking = True

def update():
    global time_of_day, network_car, network_ped, pedestrians, cars
    global contacts_cumulative, contacts_density_map
    global parking_usage_records
    global frame_count

    time_of_day.increase(Dt)
    frame_count += 1
    
    for ag in pedestrians:
        ag.act()
    
    monitor_contacts()
    
    # parking usage monitoring
    if monitoring_parking:
        time_temp = time_of_day.difference('18:00')
        if time_temp >= 0 and time_temp % 120 == 0:
            data = {n1[4:] : network_ped.nodes[n1]['capacity'] - network_ped.nodes[n1]['occupancy']
                    for n1 in possible_parking_places if network_ped.nodes[n1]['capacity'] > 0}
            lot_names = set([x.split('-')[0] for x in data])
            lot_open_spaces = {x: sum([data[xx] for xx in data if xx.split('-')[0] == x]) for x in lot_names}
            parking_usage_records[time_of_day.str()] = lot_open_spaces

In [20]:
animating = False
animation_folder = 'C:\\Users\\Hiroki\\Desktop\\animation\\'
pycxsimulator.GUI().start(func=[initialize, observe, update])

C:\Users\Hiroki\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


## Saving parking usage records

In [13]:
if monitoring_parking:
    parking_count_times = sorted(list(parking_usage_records.keys()))
    parking_lot_names = list(parking_usage_records['08:00'].keys())
    for lot in ['a1', 'b1', 'm3', 'm4', 'w', 'visitors parking']: # these were not included in the actual data
        parking_lot_names.remove(lot)
    parking_lot_names = sorted(list(set(parking_lot_names + ['visitors parking + w', 'm3 + m4'])))
    for t in parking_count_times:
        parking_usage_records[t]['visitors parking + w'] = \
            parking_usage_records[t]['visitors parking'] + parking_usage_records[t]['w']
        parking_usage_records[t]['m3 + m4'] = parking_usage_records[t]['m3'] + parking_usage_records[t]['m4']
    
    f = open('parking_usage_records-simulated.csv', 'w', newline = '')
    fcsv = csv.writer(f)
    fcsv.writerow(['lot name'] + parking_count_times)
    for lot in parking_lot_names:
        fcsv.writerow([lot] + [parking_usage_records[t][lot] for t in parking_count_times])
    f.close()

## Drawing cumulative contacts density map

In [21]:
pycxsimulator.GUI().start(func=[lambda *x:None, draw_contacts_density_map_cumulative, lambda *x:None])

C:\Users\Hiroki\Anaconda3\lib\site-packages\ipykernel_launcher.py:110: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Hiroki\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\Hiroki\Anaconda3\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "C:\Users\Hiroki\OneDrive\campus-traffic-model\pycxsimulator.py", line 232, in stepModel
    self.status.configure(foreground='black')
  File "C:\Users\Hiroki\Anaconda3\lib\tkinter\__init__.py", line 1485, in configure
    return self._configure('configure', cnf, kw)
  File "C:\Users\Hiroki\Anaconda3\lib\tkinter\__init__

## Monitoring numbers over the course of simulation

In [16]:
#  faculty, staff, GRA, UG, GD，running cars, parked cars 
numbers = {'FC':[], 'SF':[], 'GRA':[], 'UG':[], 'GD':[], 'cars moving':[], 'cars parked':[]}
initialize()
for t in range(16*60):
    if t % 30 == 0:
        print (t, end = ' ')
    update()
    for ct in ['FC', 'SF', 'GRA', 'UG', 'GD']:
        numbers[ct].append(len([x for x in pedestrians if x.category == ct and x.active and not x.left]))
    numbers['cars moving'].append(len([x for x in pedestrians if x.car != None and x.active and x.driving]))
    numbers['cars parked'].append(len([x for x in pedestrians if x.car != None and x.active and not x.driving]))

def plot_numbers():
    cla()
    for ct in numbers:
        plot(numbers[ct], label = ct)
    legend()
    
pycxsimulator.GUI().start(func=[lambda *x:None, plot_numbers, lambda *x:None])

0 30 60 90 120 150 180 210 240 270 300 330 360 390 420 450 480 510 540 570 600 630 660 690 720 750 780 810 840 870 900 930 

## Density sweep experiment

In [17]:
densities = arange(0.125, 1.01, 0.125)
num_peds = []
results = []
for density in densities:
    print(f'density = {density} : ', end='')
    num_peds_list = []
    result_list = []
    for i in range(10):
        initialize(density = density)
        while time_of_day.difference('23:00') > 0:
            if time_of_day.difference('23:00') % 30 == 0:
                print(time_of_day.str(), end=' ')
            update()
        print('')
        num_peds_list.append(len(pedestrians))
        result_list.append(contacts_cumulative)
    num_peds.append(num_peds_list)
    results.append(result_list)
    print('')

with open('density-sweep-results.csv', 'w', newline = '') as f:
    fcsv = csv.writer(f)
    fcsv.writerow(densities)
    fcsv.writerow([])
    fcsv.writerows(num_peds)
    fcsv.writerow([])
    fcsv.writerows(results)

density = 0.125 : 06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 

density = 0.625 : 06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 22:00 22:30 
06:30 07:00 07:30 08:00 08:30 09:00 09:30 10:00 10:30 11:00 11:30 12:00 12:30 13:00 13:30 14:00 14:30 15:00 15:30 16:00 16:30 17:00 17:30 18:00 18:30 19:00 19:30 20:00 20:30 21:00 21:30 